# Multilevel Bayesian Sensitivity Analysis of Upward Career Mobility

This notebook conducts a series of multilevel (mixed-effects) Bayesian logistic regression analyses to evaluate the robustness of the population-averaged estimates obtained in the main regression analysis. By introducing random effects for different grouping structures (e.g., occupation, industry, firm size), this analysis tests whether the key fixed effects in Models 1–4 remain stable after accounting for hierarchical variation.

__Important Note For Reproducibility__

* This data analysis code was developed using a proprietary, licensed commercial dataset (Lightcast data). The raw data files are NOT included in this repository and cannot be publicly shared.
* The file names for the career trajectory data used in the code are placeholders. To run this notebook, users must substitute them with their own compatible raw data files.


## Overview

The notebook uses Binomial Bayesian Generalized Linear Mixed Models (GLMM) with variational Bayes inference to efficiently fit models on a large dataset of job transitions. It systematically introduces random effects for different organizational and contextual units to:

1. Assess the stability of fixed-effect estimates (e.g., gender, race, job change type indicators) under hierarchical modeling.
2. Examine variance components across occupational, industrial, firm-size, state, and cohort groups.
3. Explore whether interaction effects (gender × job change type, race × job change type) remain credible when accounting for multi-level structures.
4. Conduct gender-stratified multilevel analyses to probe heterogeneity.

This serves as a robustness check for the main population-averaged GLM results.

## Dataset

The analysis uses the same LightCast (LC) dataset as the main analysis notebook:

* Population: 229K workers during their first 5 career years.
* Structure: Each row represents a job-to-job transition pair, with information on demographic attributes, job characteristics, and economic context.
* Random-effect grouping variables:

  * Occupation (`onet_major_x`)
  * Industry (`nacis6_major_x`)
  * State (`state_x`)
  * Career entry cohort (`job_start_year_x`)
  * Firm size group (bucketed by observed job transitions per firm)

The firm grouping variable collapses extremely sparse employers into meaningful size categories to improve model stability.

## Data Table Description: 

| Column                                         | Description                                                               |
| ---------------------------------------------- | ------------------------------------------------------------------------- |
| `max_edu_name`                                 | Highest degree attained within the first 5-year timeframe                 |
| `onet_major_x`                                 | Major SOC code (2-digit) of first job                                     |
| `naics6_major_x`                               | 2-digit NAICS sector of first job                                         |
| `company_x`                                    | First employer company name                                               |
| `state_x`                                      | First job state                                                           |
| `job_start_year_x`                             | Year of first job start                                                   |
| `num_job_changes`                              | Number of job changes in the 5-year window (top-coded at 95th percentile) |
| `gender`                                       | Gender (1 = Male, 2 = Female)                                             |
| `race`                                         | Race (1 = White, 2 = Black, 3 = Asian, 4 = Hispanic)                      |
| `generation`                                   | Estimated social generation cohort                                        |
| `state_gdp_decile_x`                           | Decile of state GDP in first job year                                     |
| `annual_state_wage_x`                          | Occupational wage for the first job (state × 6-digit occupation × year)   |
| `log_wage_x`                                   | Log-transformed first job wage                                            |
| `move_1_1`, `move_1_2`, `move_2_1`, `move_2_2` | Job change type indicators for Type-1, Type-2, Type-3, respectively       |
| `up_move`                                      | Upward mobility indicator: last job wage ≥ 5% above first job wage        |
| `firm_obs_group_x`                             | Firm size group of first job: Mega, Large, Medium, Sparse                 |


## Methodology

### Model Specification

Models are estimated using:

* BinomialBayesMixedGLM (`statsmodels.genmod.bayes_mixed_glm`)
* Fixed effects: Same specification as Models 1–4 in the main analysis, including gender, race, early career mobility types, education, wage, and regional economic controls.
* Random effects: Introduced incrementally across models to assess their impact on fixed-effect estimates.
* Continuous covariates are standardized (z-scores) prior to model fitting to improve convergence.

### Estimation

* Variational Bayes (`fit_vb`) is used for computational efficiency.
* Posterior means and standard deviations are extracted for both fixed and random effects.
* 95% credibility intervals are constructed using ±1.96 posterior SDs.
* Fixed-effect results are converted to odds ratios (OR) for interpretability.
* Random-effect variances are transformed from log(τ) to τ scale to assess the degree of heterogeneity across groups.

## Sensitivity Analyses

The notebook fits the following models:

### Single Random Effect Models (M1A–M1E)

Introduce one grouping structure at a time:

* M1A: Occupation-level random effects
* M1B: Industry-level random effects
* M1C: State-level random effects
* M1D: Career-entry cohort random effects
* M1E: Firm size group random effects

### Joint Random Effect Models (M1F–M4F)

Add multiple random effects simultaneously to assess whether fixed effects remain stable when accounting for both firm size and occupation simultaneously:

* M1F: Baseline fixed effects + firm size + occupation
* M2F: Adds gender × mobility interactions
* M3F: Adds race × mobility interactions
* M4F: Stratified by gender, with race × mobility interactions

## Outputs

For each model, the notebook:

* Prints posterior summaries to the console.
* Exports a single tidy results table (.csv) containing:

  * Posterior means and SDs for fixed effects
  * Odds ratios and 95% credibility intervals
  * Variance components for each random effect grouping variable
  * Credibility flags indicating whether intervals exclude OR = 1 (for fixed) or τ = 0 (for random)

This unified output format makes it easy to compare results across multiple robustness checks.

## Reproducibility

To replicate the sensitivity analysis:

1. Place the pre-processed Parquet dataset in the `data/` directory.
2. Open the notebook in Jupyter or JupyterLab.
3. Execute cells sequentially.
4. Review exported CSV tables for fixed and random effects.

## Dependencies

* Python 3.8+
* [pandas](https://pandas.pydata.org/)
* [numpy](https://numpy.org/)
* [statsmodels](https://www.statsmodels.org/) (Bayesian Mixed GLM module)

In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import time

# Filename breakdown: using LightCast (LC) partitions 0 to 19 (20 partitions) covering the first 5-year career of individual workers
data_filename = f'../data/lc0-19_job_pairs_y1-5_final.parquet'

def load_datatable(data_filename):
    df = pd.read_parquet(data_filename)

    # Random-effect (RE) variables: state and firm size group
    df.rename(columns={'STATE_RAW_x': 'state_x'}, inplace=True)
    
    df.rename(columns={'COMPANY_NAME_x': 'company_x'}, inplace=True)
    
    # Since # obvs per firm is extrmeley sparse (median = 2), we'll group firms into #obvs buckets
    firm_counts = df.groupby('company_x').size()
    df['per_company_obs_x'] = df['company_x'].map(firm_counts)
    
    conditions = [
        # Group 1: Sparse/Transient (N < 10) 
        (df['per_company_obs_x'] < 10),
    
        # Group 2: Medium (10 <= N <= 100)
        (df['per_company_obs_x'] >= 10) & (df['per_company_obs_x'] <= 100),
    
        # Group 3: Large (101 <= N <= 300)
        (df['per_company_obs_x'] >= 101) & (df['per_company_obs_x'] <= 300),
    
        # Group 4: Major/Mega (> 300)
        (df['per_company_obs_x'] > 300)
    ]
    
    values = [
        'Sparse_N<10',
        'Medium_N10-100',
        'Large_N101-300',
        'Mega_N>300'
    ]
     
    df['firm_obs_group_x'] = np.select(conditions, values, default='Error')
    df['firm_obs_group_x'] = df['firm_obs_group_x'].astype('category')
    
    df = df[[
        'up_move',
        'gender',
        'race',
        'onet_major_x',
        'nacis6_major_x', 
        'max_edu_name',
        'generation',
        'firm_obs_group_x', # Firm size group (RE)
        'job_start_year_x', # Career entry year (RE)
        'state_x',
        'state_gdp_decile_x',
        'num_job_changes',
        'log_wage_x',
        'move_1_1', # Type-1 job movement
        'move_1_2', # Type-2 job movement
        'move_2_1', # Type-3 job movement
    ]]
    return df

df = load_datatable(data_filename)

# Sensitivity Analyis

In [24]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
import statsmodels.formula.api as smf
import time
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# --- Fitting Binomial Bayes Mixed GLM with Variational Bayes inference given FE formula and RE formula ---

def run_mglm(fixed_formula, random_formula, output_filename, maxiter=2500, gtol=5e-4):

    # Scaling continuous variables using z-scores to improve convergence time
    mean_gdp = df['state_gdp_decile_x'].mean()
    std_gdp = df['state_gdp_decile_x'].std()
    mean_changes = df['num_job_changes'].mean()
    std_changes = df['num_job_changes'].std()
    mean_wage = df['log_wage_x'].mean()
    std_wage = df['log_wage_x'].std()
    
    df['state_gdp_decile_x_scaled'] = (df['state_gdp_decile_x'] - mean_gdp) / std_gdp
    df['num_job_changes_scaled'] = (df['num_job_changes'] - mean_changes) / std_changes
    df['log_wage_x_scaled'] = (df['log_wage_x'] - mean_wage) / std_wage
    
    model = BinomialBayesMixedGLM.from_formula(
        formula=fixed_formula,
        vc_formulas=random_formula,
        data=df
    )
    
    start = time.time()
    results = model.fit_vb(
        minim_opts={
            'maxiter': maxiter,
            'gtol': gtol
        }
    )
    end = time.time()
    print(f"Done in {end-start:.2f} sec")
    print(results.summary())

    
    # --- Fixed Effect Export  ---
    fixed_param_names = results.model.exog_names 
    fe_mean = results.fe_mean
    fe_sd = results.fe_sd
    Z_SCORE_95 = 1.96
    
    margin_of_error = Z_SCORE_95 * fe_sd
    or_lb = np.exp(fe_mean - margin_of_error)
    or_ub = np.exp(fe_mean + margin_of_error)
    
    df_fixed = pd.DataFrame({
        'Parameter': fixed_param_names, 
        'Type': 'Fixed (M)',
        'Post. Mean (Log-Odds)': fe_mean,
        'Post. SD': fe_sd,
        'Odds Ratio (OR)': np.exp(fe_mean),
        '95% CI LB (OR)': or_lb,
        '95% CI UB (OR)': or_ub,
        'Credibility': ~((or_lb < 1) & (or_ub > 1))
    })
    
    
    # --- Random Effect Export ---
    
    df_random_list = []
    re_keys = list(random_formula.keys())
    
    try:
        # Loop through each Random Effect based on the length of results.vcp_mean
        for i in range(len(results.vcp_mean)):
            re_key = re_keys[i]
            
            # Retrieve specific posterior means and SDs for the i-th RE
            vcp_mean_log_tau = results.vcp_mean[i]
            vcp_sd_log_tau = results.vcp_sd[i]
            
            # Calculate CI bounds for log(tau)
            log_tau_lb = vcp_mean_log_tau - Z_SCORE_95 * vcp_sd_log_tau
            log_tau_ub = vcp_mean_log_tau + Z_SCORE_95 * vcp_sd_log_tau
            
            # Transform CI bounds back to the tau (SD) scale
            tau_lb = np.exp(log_tau_lb)
            tau_ub = np.exp(log_tau_ub)
            
            # Use the point estimate on the tau scale:
            tau_estimate = np.exp(vcp_mean_log_tau) 
            
            # Collect data for the current RE
            df_random_list.append(pd.DataFrame({
                'Parameter': [re_key],
                'Type': ['Random (V)'],
                'Post. Mean (Log-Odds)': [tau_estimate],
                'Post. SD': [vcp_sd_log_tau], # Use SD of log(tau)
                'Odds Ratio (OR)': ['N/A'],
                '95% CI LB (OR)': [tau_lb],
                '95% CI UB (OR)': [tau_ub],
                'Credibility': [tau_lb > 0] 
            }))
            
    except IndexError:
         raise RuntimeError("Mismatch between provided random_formula keys and model results structure.")
    except Exception as e:
         raise RuntimeError(f"Failed to process Random Effects: {e}")
         
    # Concatenate all collected fixed and random effect dataframes
    if df_random_list:
        df_random = pd.concat(df_random_list, ignore_index=True)
    else:
        df_random = pd.DataFrame() # Handle case with no REs (though unlikely here)
        
    final_df = pd.concat([df_fixed, df_random], ignore_index=True)
    
    # --- Formatting and Export ---
    
    # 1. RENAME COLUMNS 
    final_df = final_df.rename(columns={
        'Post. Mean (Log-Odds)': 'Post. Mean (Log-Odds | SD/tau)',
        '95% CI LB (OR)': '95% CI LB (OR | tau)',
        '95% CI UB (OR)': '95% CI UB (OR | tau)'
    })
    
    # Identify fixed and random rows for separate formatting
    fixed_rows_mask = final_df['Type'] == 'Fixed (M)'
    random_rows_mask = final_df['Type'] == 'Random (V)'
    
    # 2. Format Fixed Effects (numeric - unchanged logic)
    numeric_cols_to_format = ['Post. Mean (Log-Odds | SD/tau)', 'Post. SD', 'Odds Ratio (OR)', '95% CI LB (OR | tau)', '95% CI UB (OR | tau)']
    
    for col in numeric_cols_to_format:
        if col in final_df.columns:
            final_df.loc[fixed_rows_mask, col] = pd.to_numeric(
                final_df.loc[fixed_rows_mask, col], errors='coerce'
            ).round(4)
            
    # 3. Format Random Effect Rows
    for index in final_df[random_rows_mask].index:
        tau_est_val = final_df.loc[index, 'Post. Mean (Log-Odds | SD/tau)']
        tau_sd_val = final_df.loc[index, 'Post. SD']
        tau_lb_val = final_df.loc[index, '95% CI LB (OR | tau)']
        tau_ub_val = final_df.loc[index, '95% CI UB (OR | tau)']
        
        # Apply string formatting to each RE row
        final_df.loc[index, 'Post. Mean (Log-Odds | SD/tau)'] = f"{tau_est_val:.4f}"
        final_df.loc[index, 'Post. SD'] = f"{tau_sd_val:.4f}"
        final_df.loc[index, 'Odds Ratio (OR)'] = 'N/A'
        final_df.loc[index, '95% CI LB (OR | tau)'] = f"{tau_lb_val:.4f}"
        final_df.loc[index, '95% CI UB (OR | tau)'] = f"{tau_ub_val:.4f}"
    
    final_df.to_csv(output_filename, index=False)
    print(f"Exported results table to {output_filename}")

## Robustness Tests for Individual RE Components 

### Mode1A: Year 1 Major SOC Groups as RE

In [106]:
df = load_datatable(data_filename)
output_filename = f'../results/mglm_m1a.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled'
)
random_formula = {
    'onet_group_re': '0 + C(onet_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 785.62 sec
                           Binomial Mixed GLM Results
                                   Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------------------
Intercept                             M    -1.1047   0.0050                      
C(gender)[T.2]                        M    -0.1534   0.0072                      
C(race)[T.2]                          M    -0.0703   0.0135                      
C(race)[T.3]                          M     0.1331   0.0140                      
C(race)[T.4]                          M     0.0234   0.0178                      
C(move_1_1)[T.1]                      M     0.9354   0.0070                      
C(move_1_2)[T.1]                      M     0.9527   0.0104                      
C(move_2_1)[T.1]                      M     0.7982   0.0114                      
C(onet_major_x)[T.13]                 M     0.4683   0.0118                      
C(onet_major_x)[T.15]    

### Model 1B: Year-1 Major NAICS Groups as RE

In [249]:
df = load_datatable(data_filename)
output_filename = f'data/mglm_m1b.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled'
)
random_formula = {
    'naics_group_re': '0 + C(nacis6_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 817.64 sec
                           Binomial Mixed GLM Results
                                   Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------------------
Intercept                             M    -1.1046   0.0051                      
C(gender)[T.2]                        M    -0.1534   0.0072                      
C(race)[T.2]                          M    -0.0703   0.0135                      
C(race)[T.3]                          M     0.1331   0.0140                      
C(race)[T.4]                          M     0.0234   0.0178                      
C(move_1_1)[T.1]                      M     0.9354   0.0070                      
C(move_1_2)[T.1]                      M     0.9527   0.0104                      
C(move_2_1)[T.1]                      M     0.7982   0.0114                      
C(onet_major_x)[T.13]                 M     0.4683   0.0118                      
C(onet_major_x)[T.15]    

### Model 1C: Year-1 States as RE

In [242]:
df = load_datatable(data_filename)
output_filename = f'../results/mglm_m1c.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(state_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled'
)
random_formula = {
    'state_group_re': '0 + C(state_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 1562.55 sec
                           Binomial Mixed GLM Results
                                   Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------------------
Intercept                             M    -0.9937   0.0051                      
C(gender)[T.2]                        M    -0.1566   0.0072                      
C(race)[T.2]                          M    -0.0690   0.0135                      
C(race)[T.3]                          M     0.1302   0.0140                      
C(race)[T.4]                          M     0.0217   0.0178                      
C(move_1_1)[T.1]                      M     0.9333   0.0070                      
C(move_1_2)[T.1]                      M     0.9529   0.0104                      
C(move_2_1)[T.1]                      M     0.7991   0.0114                      
C(onet_major_x)[T.13]                 M     0.4416   0.0118                      
C(onet_major_x)[T.15]   

### Model 1D: Year 1 Cohort as RE

In [259]:
df = load_datatable(data_filename)
output_filename = f'data/mglm_m1d.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(job_start_year_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled'
)
random_formula = {
    'job_start_year_group_re': '0 + C(job_start_year_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 1021.39 sec
                           Binomial Mixed GLM Results
                                   Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
---------------------------------------------------------------------------------
Intercept                             M    -1.3167   0.0051                      
C(gender)[T.2]                        M    -0.1546   0.0072                      
C(race)[T.2]                          M    -0.0730   0.0135                      
C(race)[T.3]                          M     0.1288   0.0140                      
C(race)[T.4]                          M     0.0192   0.0178                      
C(move_1_1)[T.1]                      M     0.9316   0.0070                      
C(move_1_2)[T.1]                      M     0.9492   0.0104                      
C(move_2_1)[T.1]                      M     0.7974   0.0114                      
C(onet_major_x)[T.13]                 M     0.4504   0.0118                      
C(onet_major_x)[T.15]   

### Model 1E: Year 1 Firm Size as RE

In [25]:
df = load_datatable(data_filename)
output_filename = f'../results/mglm_m1e.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(firm_obs_group_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled'
)
random_formula = {
    'firm_obs_group_re': '0 + C(firm_obs_group_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 780.24 sec
                             Binomial Mixed GLM Results
                                      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
------------------------------------------------------------------------------------
Intercept                                M    -1.0581   0.0051                      
C(gender)[T.2]                           M    -0.1520   0.0072                      
C(race)[T.2]                             M    -0.0717   0.0135                      
C(race)[T.3]                             M     0.1270   0.0140                      
C(race)[T.4]                             M     0.0236   0.0178                      
C(move_1_1)[T.1]                         M     0.9409   0.0070                      
C(move_1_2)[T.1]                         M     0.9483   0.0104                      
C(move_2_1)[T.1]                         M     0.8054   0.0114                      
C(onet_major_x)[T.13]                    M     0.4557   0.0118             

## Robustness Tests for Joint Random Effects

### Model 1F: Year 1 Firm Size + Occupation as REs

In [26]:
df = load_datatable(data_filename)
output_filename = f'data/mglm_m1f.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(firm_obs_group_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled'
)
random_formula = {
    'firm_obs_group_re': '0 + C(firm_obs_group_x)',
    'onet_group_re': '0 + C(onet_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 969.74 sec
                             Binomial Mixed GLM Results
                                      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
------------------------------------------------------------------------------------
Intercept                                M    -1.0580   0.0051                      
C(gender)[T.2]                           M    -0.1520   0.0072                      
C(race)[T.2]                             M    -0.0717   0.0135                      
C(race)[T.3]                             M     0.1270   0.0140                      
C(race)[T.4]                             M     0.0236   0.0178                      
C(move_1_1)[T.1]                         M     0.9409   0.0070                      
C(move_1_2)[T.1]                         M     0.9484   0.0104                      
C(move_2_1)[T.1]                         M     0.8054   0.0114                      
C(onet_major_x)[T.13]                    M     0.4557   0.0118             

### Model 2F: Year 1 Firm Size + Occupation as REs

In [27]:
df = load_datatable(data_filename)
output_filename = f'../results/mglm_m2f.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(firm_obs_group_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled' +
'+ C(gender):C(move_1_1) + C(gender):C(move_1_2) + C(gender):C(move_2_1)'
)
random_formula = {
    'firm_obs_group_re': '0 + C(firm_obs_group_x)',
    'onet_group_re': '0 + C(onet_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 993.91 sec
                             Binomial Mixed GLM Results
                                      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
------------------------------------------------------------------------------------
Intercept                                M    -1.0958   0.0051                      
C(gender)[T.2]                           M    -0.0770   0.0072                      
C(race)[T.2]                             M    -0.0718   0.0135                      
C(race)[T.3]                             M     0.1274   0.0140                      
C(race)[T.4]                             M     0.0238   0.0178                      
C(move_1_1)[T.1]                         M     0.9692   0.0070                      
C(move_1_2)[T.1]                         M     1.0005   0.0104                      
C(move_2_1)[T.1]                         M     0.8560   0.0114                      
C(onet_major_x)[T.13]                    M     0.4543   0.0118             

### Model 3F: Year 1 Firm Size + Occupation as REs

In [29]:
df = load_datatable(data_filename)
output_filename = f'data/mglm_m3f.csv'
fixed_formula = (
'up_move ~ C(gender) + C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(firm_obs_group_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled' +
'+ C(race):C(move_1_1) + C(race):C(move_1_2) + C(race):C(move_2_1)'
)
random_formula = {
    'firm_obs_group_re': '0 + C(firm_obs_group_x)',
    'onet_group_re': '0 + C(onet_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 1039.41 sec
                             Binomial Mixed GLM Results
                                      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
------------------------------------------------------------------------------------
Intercept                                M    -1.0669   0.0051                      
C(gender)[T.2]                           M    -0.1518   0.0072                      
C(race)[T.2]                             M    -0.0652   0.0136                      
C(race)[T.3]                             M     0.1699   0.0139                      
C(race)[T.4]                             M     0.0632   0.0178                      
C(move_1_1)[T.1]                         M     0.9446   0.0070                      
C(move_1_2)[T.1]                         M     0.9647   0.0104                      
C(move_2_1)[T.1]                         M     0.8274   0.0114                      
C(onet_major_x)[T.13]                    M     0.4560   0.0118            

### Model 4F: Year 1 Firm Size + Occupation as REs

In [43]:
df = load_datatable(data_filename)
df = df.query("gender==1")
output_filename = f'../results/mglm_m4f-m.csv'
fixed_formula = (
'up_move ~ C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(firm_obs_group_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled' +
'+ C(race):C(move_1_1) + C(race):C(move_1_2) + C(race):C(move_2_1)'
)
random_formula = {
    'firm_obs_group_re': '0 + C(firm_obs_group_x)',
    'onet_group_re': '0 + C(onet_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 385.27 sec
                             Binomial Mixed GLM Results
                                      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
------------------------------------------------------------------------------------
Intercept                                M    -1.2127   0.0071                      
C(race)[T.2]                             M    -0.0955   0.0205                      
C(race)[T.3]                             M     0.0898   0.0196                      
C(race)[T.4]                             M     0.0343   0.0253                      
C(move_1_1)[T.1]                         M     0.9343   0.0101                      
C(move_1_2)[T.1]                         M     0.9819   0.0146                      
C(move_2_1)[T.1]                         M     0.8564   0.0162                      
C(onet_major_x)[T.13]                    M     0.5231   0.0168                      
C(onet_major_x)[T.15]                    M     0.4898   0.0191             

In [44]:
df = load_datatable(data_filename)
df = df.query("gender==2")
output_filename = f'data/mglm_m4f-f.csv'
fixed_formula = (
'up_move ~ C(race) + C(move_1_1) + C(move_1_2) + C(move_2_1)' +
'+ C(onet_major_x) + C(nacis6_major_x) + C(firm_obs_group_x) + C(max_edu_name) + C(generation)' +
'+ state_gdp_decile_x_scaled + num_job_changes_scaled + log_wage_x_scaled' +
'+ C(race):C(move_1_1) + C(race):C(move_1_2) + C(race):C(move_2_1)'
)
random_formula = {
    'firm_obs_group_re': '0 + C(firm_obs_group_x)',
    'onet_group_re': '0 + C(onet_major_x)'
}
run_mglm(fixed_formula, random_formula, output_filename)

Done in 371.29 sec
                             Binomial Mixed GLM Results
                                      Type Post. Mean Post. SD   SD  SD (LB) SD (UB)
------------------------------------------------------------------------------------
Intercept                                M    -1.0501   0.0072                      
C(race)[T.2]                             M    -0.0504   0.0181                      
C(race)[T.3]                             M     0.2672   0.0197                      
C(race)[T.4]                             M     0.0953   0.0250                      
C(move_1_1)[T.1]                         M     0.9558   0.0098                      
C(move_1_2)[T.1]                         M     0.9445   0.0148                      
C(move_2_1)[T.1]                         M     0.7925   0.0161                      
C(onet_major_x)[T.13]                    M     0.3818   0.0167                      
C(onet_major_x)[T.15]                    M     0.2843   0.0302             